In [21]:
import re
import urllib.request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import random

import unicodedata
import unicodeblock.blocks

import os

base_dir = 'data/현대 국어 사용 빈도 조사 2'

sorted(os.listdir(base_dir))

['구통계.txt',
 '구통계.xlsx',
 '어미통계.txt',
 '어미통계.xlt',
 '음절통계.txt',
 '음절통계.xlt',
 '일반어휘통계.txt',
 '일반어휘통계.xlsx',
 '자모통계.txt',
 '자모통계.xlt',
 '조사통계.txt',
 '조사통계.xlt',
 '현대 국어 사용 빈도 조사 2(2005).PDF']

In [20]:
Phrases = pd.read_excel(os.path.join(base_dir, '구통계.xlsx')) #.sort_values('빈도', ascending=False)[:50]
Vocab = pd.read_excel(os.path.join(base_dir, '일반어휘통계.xlsx')) #.sort_values('빈도', ascending=False)[:50]
Letters = pd.read_excel(os.path.join(base_dir, '자모통계.xlt')) #.sort_values('빈도', ascending=False)[:20]
Auxiliaries = pd.read_excel(os.path.join(base_dir, '조사통계.xlt')) #.sort_values('빈도', ascending=True)[:20]
Syllables = pd.read_excel(os.path.join(base_dir, '음절통계.xlt')) #.sort_values('빈도', ascending=True)[:20]

In [28]:
Vocab['품사'].value_counts()

명    58605
동    13292
형     3411
부     3103
관     2570
의      542
감      384
수      285
대      162
접       76
보       66
지        5
Name: 품사, dtype: int64

In [64]:
Vocab[(Vocab.품사 == '부')&(Vocab.빈도 > 10)].sample(10).sort_values('빈도')

,순위,빈도,어휘,풀이,품사
69629,1160,12,총총히04,悤悤-,부
67214,1159,13,착실히,着實-,부
11601,1157,15,끌끌01,혀 차는 소리,부
59395,1145,27,저만큼,NaN,부
77145,1127,45,하루빨리,NaN,부
77079,1112,60,하나하나,NaN,부
12211,1106,66,날로01,날이 갈수록,부
17054,1105,67,도리어,NaN,부
49635,1054,118,오로지,NaN,부
15598,937,235,대개03,大槪,부


In [25]:
Phrases[Phrases['빈도'] == 12].sample(10)

,순위,빈도,항목,풀이
4109,70,12,장기 이식,臟器移植
3290,70,12,연립 주택,聯立住宅
2435,70,12,상임 위원회,常任委員會
3558,70,12,원시 시대,原始時代
5257,70,12,팝 아트,pop art
897,70,12,금융 자산,金融資産
5016,70,12,칸 영화제,Cannes映畫祭
4404,70,12,정치 발전,政治發展
4236,70,12,전송 속도,傳送速度
2564,70,12,선거 기간,選擧期間
